In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, BatchNormalization, MaxPooling2D, LeakyReLU, ReLU, PReLU
from tensorflow.keras.optimizers import RMSprop, Nadam, Adadelta, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#Input data preprocessing
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))
#Output data preprocessing
y_train = to_categorical(y_train, 10) # one-hot encoding, 1차원 -> 2차원
y_test = to_categorical(y_test, 10)

print(f"X_train_shape: {X_train.shape}") 
print(f"X_test_shape: {X_test.shape}")
print(f"y_train_shape: {y_train.shape}")
print(f"y_test_shape: {y_test.shape}")  

X_train_shape: (60000, 28, 28, 1)
X_test_shape: (10000, 28, 28, 1)
y_train_shape: (60000, 10)
y_test_shape: (10000, 10)


In [ ]:
datagen_train = ImageDataGenerator(rotation_range = 10, 
                                   width_shift_range = 0.25, 
                                   height_shift_range = 0.25, 
                                   shear_range = 0.1,  
                                   zoom_range = 0.4,
                                   horizontal_flip = False) 

datagen_val = ImageDataGenerator() 
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau( 
    monitor='loss',    
    factor=0.25,       
    patience=2,        
    verbose=1,         
    mode="auto",       
    min_delta=0.0001,  
    cooldown=0,        
    min_lr=0.00001     
    )

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(64,  (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(64,  (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(128, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(128, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),    
    
    tf.keras.layers.Conv2D(256, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(256, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),##
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(alpha=0.1),
 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 28, 28, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 28, 28, 64)        0

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300, restore_best_weights=True)
history = model.fit(datagen_train.flow(X_train, y_train, batch_size=256),
                              steps_per_epoch=len(X_train)//256,
                              epochs=50,
                              validation_data=(X_test, y_test),
                              callbacks=[learning_rate_reduction, es],
                              verbose=2)

Epoch 1/50
234/234 - 42s - loss: 0.4480 - accuracy: 0.8558 - val_loss: 0.4586 - val_accuracy: 0.9703 - lr: 0.0010 - 42s/epoch - 180ms/step
Epoch 2/50
234/234 - 37s - loss: 0.1554 - accuracy: 0.9522 - val_loss: 0.0567 - val_accuracy: 0.9855 - lr: 0.0010 - 37s/epoch - 159ms/step
Epoch 3/50
234/234 - 37s - loss: 0.1155 - accuracy: 0.9636 - val_loss: 0.0707 - val_accuracy: 0.9814 - lr: 0.0010 - 37s/epoch - 158ms/step
Epoch 4/50
234/234 - 39s - loss: 0.0999 - accuracy: 0.9685 - val_loss: 0.0287 - val_accuracy: 0.9916 - lr: 0.0010 - 39s/epoch - 166ms/step
Epoch 5/50
234/234 - 37s - loss: 0.0942 - accuracy: 0.9705 - val_loss: 0.0290 - val_accuracy: 0.9913 - lr: 0.0010 - 37s/epoch - 158ms/step
Epoch 6/50
234/234 - 37s - loss: 0.0834 - accuracy: 0.9742 - val_loss: 0.0224 - val_accuracy: 0.9932 - lr: 0.0010 - 37s/epoch - 158ms/step
Epoch 7/50
234/234 - 37s - loss: 0.0829 - accuracy: 0.9740 - val_loss: 0.0293 - val_accuracy: 0.9910 - lr: 0.0010 - 37s/epoch - 157ms/step
Epoch 8/50
234/234 - 39s - 

In [ ]:
model.evaluate(X_test, y_test, batch_size=256)

40/40 [==============================] - 2s 48ms/step - loss: 0.0115 - accuracy: 0.9962


[0.011515125632286072, 0.9962000250816345]